# Searching and Retrieving Vector Tables

__________________

The Descartes Labs API also supports spatial and aspatial tabular datasets. 

This guide covers the basic methods for searching, filtering, joining, and collecting tabular data. For full reference visit the [Vector API Documentation page](https://docs.descarteslabs.com/api/vector.html).

__Note__
Currently Vector is pre-installed on Workbench, however is not included with the main Descartes Labs Python client. To install Vector, run the following:

    !pip install descarteslabs-vector

In [ ]:
import descarteslabs as dl
from descarteslabs.catalog import properties as p

In [ ]:
from descarteslabs.vector import Table

## Tablular Data
The foundational class in Vector is the `Table`. As with other objects such as products and blobs, you can retrieve a table by it's *unique* ID. 

Let's start by taking a look at the table for US county and county-equivalents and inspect both the number of rows and the column names:

In [ ]:
counties_table = Table.get("descarteslabs:hifld:us-counties")

print(f"Columns: {counties_table.columns}")
print(f"Count: {counties_table.count()}")
counties_table

List all tables you have access to for their table ID:

    for table in Table.list():
        print(table.id)

## Filtering Tables

Vector is much more than a simple store for tabular data. Much of Catalog's search, intersect, and filter functionality is exposed as well. 

There are a few ways to pass spatial or attribute filters to tables, all through the table's __aoi__ and __property_filter__: 
* Modify the `TableOptions` directly, or
* The easiest way is to pass __aoi__ or __property_filter__ arguments upon table creation 

Here's an example of modifying our table's __property_filter__ options, finding all counties in the US state of Washington by their STATEFP attribute:

In [ ]:
counties_table.options.property_filter = p.STATEFP == "53"
print(f"Counties in Washington: {counties_table.count()}")

The same could be achieved on class instantiation. We can also pass __columns__ to restrict the collected columns:

In [ ]:
counties_table = Table.get(
    "descarteslabs:hifld:us-counties",
    columns=["STATEFP", "NAME", "geometry"],
    property_filter=(p.STATEFP == "53"),
)
print(f"Counties in Washington: {counties_table.count()}")
print(f"Columns: {counties_table.options.columns}")

## Retrieving Data
We can retrieve the contents of our table at any point as a geopandas geodataframe by calling `Table.collect()`:

In [ ]:
counties_gdf = counties_table.collect()
print(counties_gdf.shape)
print(counties_gdf.columns)
counties_gdf.plot(figsize=(5, 5))

Note that the __columns__ we filtered above were returned in our dataframe, and not all possible fields.

## Spatial Intersections
We can also implement a spatial __aoi__ filter. 

How about we retrieve the US country boundary by passing an AOI to our country boundaries table:

In [ ]:
countries_table = Table.get(
    "descarteslabs:lsib-boundaries", aoi=counties_gdf["geometry"][0]
)
countries_gdf = countries_table.collect()
countries_gdf

## Joining Tables
Vector perform spatial and aspatial joins as well. 

In the below example we will find counties in Washington that intersect a census tract with a high avalanche risk score. 

To start, we will filter all the census tracts from the [National Risk Index](https://www.fema.gov/flood-maps/products-tools/national-risk-index) table with an avalanche risk score greater than 50 through a __property_filter__:

In [ ]:
nri_table = Table.get(
    "descarteslabs:nri:tracts",
    columns=["uuid", "NRI_ID", "AVLN_RISKS", "geometry"],
    property_filter=(p.AVLN_RISKS > 50),
)
nri_table.count()

We could have also modified the table's options directly just as well:

    nri_table = Table.get("descarteslabs:nri:tracts")
    nri_table.options.property_filter = (p.AVLN_RISKS>80)

A spatial join is as simple as calling `Table.sjoin` and passing a __join_type__:

In [ ]:
avln_join_gdf = counties_table.sjoin(join_table=nri_table, join_type="INTERSECTS")

We can now inspect the geodataframe to see how the associated joined data is returned:

In [ ]:
avln_join_gdf.head(5)

Note that each field in the resulting joined table has the following naming structure:

    {table-name}.{field-name}
    
Let's overlay our high-risk counties with the rest of the state:

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
counties_gdf.plot(ax=ax, facecolor="none", edgecolor="k")
avln_join_gdf.plot(ax=ax, column="tracts.AVLN_RISKS", cmap="winter", legend=True)
ax.set_title("Avalanche Risk");

## Visualization
We can also visualize our vector data on a leaflet map, such as through Dynamic Compute:

In [ ]:
import descarteslabs.dynamic_compute as dc

In [ ]:
m = dc.map
m.center = 47.4578, -121.6956
zoom = 6

In [ ]:
counties_table.visualize(name="Washington", map=m)

In [ ]:
m